## 4. Reflection(langchain)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from dotenv import load_dotenv
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [3]:
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
except Exception as e:
    print(f"언어 모델 초기화 오류: {e}")
    llm = None

### 1. 루프 구현

In [4]:
def run_reflection_loop():
    """
    파이썬 함수를 점진적으로 개선하기 위한 다단계 AI 리플렉션 루프를 시연합니다.
    """
    # --- 핵심 작업 ---
    task_prompt = """
    귀하의 작업은 `calculate_factorial`이라는 이름의 파이썬 함수를 생성하는 것입니다.
    이 함수는 다음을 수행해야 합니다:
    - 정수를 입력으로 받습니다.
    - 그 정수의 계승을 반환합니다.
    - 함수를 설명하는 독스트링을 포함합니다.
    - 입력이 0인 경우의 엣지 케이스를 처리합니다 (0의 계승은 1입니다).
    - 음수 입력에 대해 ValueError를 발생시킵니다.
    """

    # --- 메시지 기록 초기화 ---
    message_history = [
        SystemMessage(content="귀하는 전문 파이썬 개발자입니다. 작업 지침을 정확히 따르세요."),
        HumanMessage(content=task_prompt)
    ]

    # --- 리플렉션 루프 ---
    for i in range(3):  # 무한 루프를 피하기 위해 3회 반복으로 제한
        # --- 1. 생성/정제 단계 ---
        if i == 0:
            # 첫 번째 반복: 초기 코드 생성
            print("\n>>> 단계 1: 초기 코드 생성 중...")
            response = llm.invoke(message_history)
            current_code = response.content
        else:
            # 후속 반복: 이전 비판을 기반으로 정제
            print(f"\n>>> 단계 1: 코드 정제 중 (반복 {i+1})...")
            message_history.append(HumanMessage(content="제공된 비판을 사용하여 코드를 정제하세요."))
            response = llm.invoke(message_history)
            current_code = response.content

        print("\n--- 생성된 코드 (v" + str(i + 1) + ") ---\n" + current_code)
        message_history.append(response)  # 생성된 코드를 기록에 추가

        # --- 2. 리플렉션 단계 ---
        print("\n>>> 단계 2: 생성된 코드에 대한 리플렉션...")
        # 리플렉션 에이전트를 위한 특정 프롬프트 생성.
        # 이는 모델이 수석 코드 검토자로 행동하도록 요청합니다.
        reflector_prompt = [
            SystemMessage(content="""
                귀하는 수석 소프트웨어 엔지니어이자 파이썬 전문가입니다.
                귀하의 역할은 세심한 코드 검토를 수행하는 것입니다.
                제공된 파이썬 코드를 원래 작업 요구사항에 기반하여 비판적으로 평가하세요.
                버그, 스타일 문제, 누락된 엣지 케이스, 개선 영역을 찾으세요.
                코드가 완벽하고 모든 요구사항을 충족하면, 단일 구문 'CODE_IS_PERFECT'로 응답하세요.
                그렇지 않으면, 비판의 글머리 기호 목록을 제공하세요.
            """),
            HumanMessage(content=f"원래 작업:\n{task_prompt}\n\n검토할 코드:\n{current_code}")
        ]

        critique_response = llm.invoke(reflector_prompt)
        critique = critique_response.content

        # --- 중지 조건 ---
        if "CODE_IS_PERFECT" in critique:
            print("\n--- 비판 ---\n더 이상 비판이 없습니다. 코드가 만족스럽습니다.")
            break

        print("\n--- 비판 ---\n" + critique)
        # 다음 정제 루프를 위한 기록에 비판 추가.
        message_history.append(HumanMessage(content=f"이전 코드의 비판:\n{critique}"))

    print("\n" + "="*30 + " 최종 결과 " + "="*30)
    print("\n리플렉션 프로세스 후 최종 정제 코드:\n")
    print(current_code)

In [5]:
run_reflection_loop()


>>> 단계 1: 초기 코드 생성 중...

--- 생성된 코드 (v1) ---
아래는 요청하신 `calculate_factorial` 함수의 구현입니다. 이 함수는 정수를 입력으로 받아 해당 정수의 계승을 계산하고, 0의 계승은 1로 처리하며, 음수 입력에 대해서는 `ValueError`를 발생시킵니다.

```python
def calculate_factorial(n):
    """
    주어진 정수 n의 계승을 계산하여 반환합니다.

    계승은 n!로 표기되며, n이 0일 경우 1을 반환합니다.
    음수 입력에 대해서는 ValueError를 발생시킵니다.

    매개변수:
    n (int): 계승을 계산할 정수

    반환값:
    int: n의 계승
    """
    if n < 0:
        raise ValueError("음수에 대한 계승은 정의되지 않습니다.")
    elif n == 0:
        return 1
    else:
        factorial = 1
        for i in range(1, n + 1):
            factorial *= i
        return factorial
```

이 함수는 다음과 같은 방식으로 작동합니다:
- 입력이 음수일 경우 `ValueError`를 발생시킵니다.
- 입력이 0일 경우 1을 반환합니다.
- 그 외의 경우에는 반복문을 사용하여 계승을 계산합니다.

>>> 단계 2: 생성된 코드에 대한 리플렉션...

--- 비판 ---
- 코드의 전반적인 구조와 기능은 요구사항을 충족하고 있습니다. 그러나 몇 가지 개선할 점이 있습니다.
- 독스트링은 잘 작성되어 있으나, 반환값에 대한 설명이 조금 더 명확하게 작성될 수 있습니다. 예를 들어, "n의 계승" 대신 "n의 계승(n!)"이라고 명시하면 더 좋습니다.
- 음수 입력에 대한 예외 처리는 잘 되어 있으나, 입력이 정수가 아닐 경우에 대한 예외 처리가 없습니다. `TypeError`를

코드는 환경 설정, API 키 로드, GPT-4o와 같은 강력한 언어 모델을 낮은 온도로 초기화하여 시작합니다.   
핵심 작업은 숫자의 계승을 계산하는 파이썬 함수를 요청하는 프롬프트로 정의되며, 독스트링, 엣지 케이스(0의 계승), 음수 입력에 대한 오류 처리에 대한 특정 요구사항을 포함합니다.   
run_reflection_loop 함수는 반복 정제 프로세스를 오케스트레이션합니다.   
루프 내에서, 첫 번째 반복에서 언어 모델은 작업 프롬프트를 기반으로 초기 코드를 생성합니다.   
후속 반복에서, 이전 단계의 비판을 기반으로 코드를 정제합니다.   
별도의 "리플렉션" 역할, 언어 모델이지만 다른 시스템 프롬프트로, 수석 소프트웨어 엔지니어로 행동하여 원래 작업 요구사항에 대해 생성된 코드를 비판합니다.   
이 비판은 문제 목록의 글머리 기호로 제공되거나 문제가 없으면 'CODE_IS_PERFECT' 구문입니다. 루프는 비판이 코드가 완벽하다고 나타내거나 최대 반복 횟수에 도달할 때까지 계속됩니다.   
대화 기록은 각 단계에서 언어 모델에 전달되어 생성/정제 및 리플렉션 단계 모두에 맥락을 제공합니다.   
마지막으로, 스크립트는 루프가 끝난 후 마지막 생성된 코드 버전을 출력합니다.